## IMDB review sentiment Analysis


#### Μέλη Ομάδας:
- Ευάγγελος Λευτάκης : 3200093
- Ρέα Σκλήκα : 3210181
- Σοφία-Ζωή Σωτηρίου : 3210192

### Dependencies

In [ ]:
pip install pandas
pip install numpy
pip install scipy
pip install tensorflow
pip install sklearn

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn

### 1st Part

First We Prepare the train and test data and create a binary representation of it.

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

(x_train_imdb, y_train), (x_test_imdb, y_test) = tf.keras.datasets.imdb.load_data()


# x_train_imdb, y_train_imdb = temp_x_train_imdb[:split_index], y_train[:split_index]
# x_dev_imdb, y_dev_imdb = temp_x_train_imdb[split_index:], y_train[split_index:]

word_index = tf.keras.datasets.imdb.get_word_index()
index2word = dict((i + 3, word) for (word, i) in word_index.items())
index2word[0] = '[pad]'
index2word[1] = '[bos]'
index2word[2] = '[oov]'
x_train_imdb = np.array([' '.join([index2word[idx] for idx in text]) for text in x_train_imdb])
x_test_imdb = np.array([' '.join([index2word[idx] for idx in text]) for text in x_test_imdb])

binary_vectorizer = CountVectorizer(binary=True, min_df=100)
x_train = binary_vectorizer.fit_transform(x_train_imdb)
x_test = binary_vectorizer.transform(x_test_imdb)
print(
    'Vocabulary size:', len(binary_vectorizer.vocabulary_)
)
x_train = x_train.astype(int)
x_test = x_test.astype(int)
# print(x_train.shape)



Vocabulary size: 3834


#### Logistic Regression:


In [40]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.sparse import issparse

class LogisticRegression():
   

    def __init__(self,epochs=20,learning_rate=0.001,threshold=0.5, regularization_factor=1):
        self.epochs=epochs;
        self.learning_rate=learning_rate
        self.weights = np.array([])
        self.threshold=threshold
        self.regularization_factor = regularization_factor
        
    
    def set_threshold(self,value):
        self.threshold = value
    def sigmoid(self , t):
        return 1/(1+np.exp(-t))


    def fit(self,x_train_input,y_train_input):
        ''' '''
        feature_vector_length = x_train_input.shape[1]
        split_index = int(0.8 * x_train_input.shape[0])  # 80% for training, 20% for dev
        x_train, y_train = x_train_input[:split_index], y_train_input[:split_index]
        x_dev, y_dev= x_train_input[split_index:], y_train_input[split_index:]
        x_dev = x_dev.toarray()

        x_train = x_train.toarray()
        ones_column = np.ones(x_train.shape[0])
        x_train = np.insert(x_train, 0, ones_column, axis=1)
        self.weights = self.initialize_weights(feature_vector_length)
        
        for epoch in range(self.epochs):
            for i in range(x_train.shape[0]):
                x_i = x_train[i-1]
                y_i = y_train[i-1]

                self.update_weights(x_i,y_i)
        y_dev_predicted = self.predict(x_dev)
        return self.evaluate(y_dev_predicted, y_dev)


        
    
    def predict(self, x_test):
        
        features_length = x_test.shape[1]
        if issparse(x_test):
            x_test = x_test.toarray()
        ones_column = np.ones(x_test.shape[0])
        x_test = np.insert(x_test, 0, ones_column, axis=1)#insert 1 in x_vector for w0

        y_test_predicted = []


        for test_case in x_test:
            probability_positive =  self.sigmoid(np.dot(self.weights,test_case))
            predicted_class = 1 if probability_positive >= self.threshold else 0
            y_test_predicted.append(predicted_class)
        return np.array(y_test_predicted)



    @staticmethod
    def initialize_weights(size):
        '''We Initialize the weights with random values with a mean of 0 and a standard deviation of 0.01
          This is a common practice to prevent the weights from being too large at the begining '''
        return np.random.randn(size+1) * 0.01

    def update_weights(self,x_test,y_test):
        predicted_prob = self.sigmoid(np.dot(self.weights,x_test))
        gradient = ((y_test - predicted_prob) * x_test)
        self.weights += self.learning_rate * gradient 
    
    def evaluate(self, y_true, y_predicted):
        accuracy = accuracy_score(y_true, y_predicted)
        print("Accuracy:", accuracy)

        precision = precision_score(y_true, y_predicted)
        print("Precision:", precision)

        # Compute recall
        recall = recall_score(y_true, y_predicted)
        print("Recall:", recall)

        # Compute F1 score
        f1 = f1_score(y_true, y_predicted)
        print("F1 Score:", f1)
        print("Threshold: "+str(self.threshold))
        return np.array([precision,recall])
    

    def generate_pr_curve(self, x_train_input,y_train_input,size=20):
        currentThreshold = 0
        percisions=[]
        recalls=[]
        for i in range(size):
            currentThreshold += 1./size
            if (currentThreshold>0.3 and currentThreshold<0.75):
                self.set_threshold(currentThreshold)
                fit_pr = self.fit(x_train_input , y_train_input)
                percisions.append(fit_pr[0])
                recalls.append(fit_pr[1])
        print(percisions)
        plt.plot(recalls,percisions ,label='Precision-Recall Curve')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision-Recall Diagram')
        plt.legend()
        plt.show()


#### Results
Here we try out the Logistic Regression learning algorithm

In [42]:
LogReg = LogisticRegression(100,threshold=0.5, regularization_factor=0.01)
LogReg.fit(x_train,y_train)
y_predicted = LogReg.predict(x_test)
# fit(x_train,y_train)
 
# y_predicted = LogReg.predict(x_test)

LogReg.evaluate(y_test,y_predicted)

Accuracy: 0.8764
Precision: 0.880113406237343
Recall: 0.8709418837675351
F1 Score: 0.8755036261079775
Threshold: 0.5
Accuracy: 0.87116
Precision: 0.8669039145907473
Recall: 0.87696
F1 Score: 0.8719029628156691
Threshold: 0.5


array([0.86690391, 0.87696   ])

#### Naive Bayes:

In [14]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
#from scipy.sparse import isspars

class NaiveBayes: 
    def __init__(self):
            self.probabilityC1= None #probabilityC0 = 1 -probabilityC1
            self.boundProbabilityC0 = None 
            self.boundProbabilityC1 = None

    def fit(self, x_train_input, y_train_input): 
        split_index = int(0.8 * x_train_input.shape[0])  # 80% for training, 20% for dev
        x_train, y_train = x_train_input[:split_index], y_train_input[:split_index]
        x_dev, y_dev= x_train_input[split_index:], y_train_input[split_index:]

        numberOfExamples = x_train.shape[0]
        numberOfFeatures = x_train.shape[1]
        x_train = x_train.toarray()
        #y_train = y_train.flatten()

        positiveReviews = 0
        for label in y_train: 
            if label == 1: 
                positiveReviews += 1 

        self.probabilityC1 = positiveReviews / len(y_train)

        '''
        C0: P( X(i) = 0 | C = 0) and C1 = P ( X(i) = 0 | C = 1)
        P( X(i) = 1 | C = 0) = 1 - P( X(i) = 0 | C = 0) and P( X(i) = 1 | C = 1) = P( X(i) = 0 | C = 1)
        '''
        self.boundProbabilityC0 = np.zeros(numberOfFeatures)  
        self.boundProbabilityC1 = np.zeros(numberOfFeatures)
        for i in range(numberOfFeatures): 

            #For every word that is not on a review add to the array correspnding to the review result of the example 
            for j in range(numberOfExamples):  
                if y_train[j] == 0 and x_train[j][i] == 0: 
                    self.boundProbabilityC0[i] += 1 
                elif y_train[j] == 1 and x_train[j][i] == 0: 
                    self.boundProbabilityC1[i] += 1 
        """
        Adding Laplace estimator with an alpha value of 1 
        """
        self.boundProbabilityC0 = [x + 1 / (numberOfExamples - positiveReviews + x) for x in self.boundProbabilityC0] 
        self.boundProbabilityC1 = [x + 1 / (positiveReviews+ x) for x in self.boundProbabilityC1]

    def predict(self, x_test, y_test):
        prediction = list()
        c1 = self.boundProbabilityC1
        c0 = self.boundProbabilityC0
        numberOfFeatures = x_test.shape[1]
        x_test = x_test.toarray()

        for x in range(x_test.shape[0]): 
            positiveReviewProbability = self.probabilityC1 
            negativeReviewProbability = (1 - self.probabilityC1)

            for y in range(numberOfFeatures):
                if x_test[x][y] == 0 and y_test[x] == 0: 
                    negativeReviewProbability = round(negativeReviewProbability * self.boundProbabilityC0[y], 2)
                elif  x_test[x][y] == 1 and y_test[x] == 0: 
                    negativeReviewProbability = round(negativeReviewProbability * (1 - self.boundProbabilityC0[y]), 2)
                elif  x_test[x][y] == 0 and y_test[x] == 1:
                    positiveReviewProbability = round(positiveReviewProbability * self.boundProbabilityC1[y], 2)
                elif  x_test[x][y] == 1 and y_test[x] == 1:
                    positiveReviewProbability = round(positiveReviewProbability * (1 - self.boundProbabilityC1[y]), 2)

            if positiveReviewProbability > negativeReviewProbability: 
                prediction.append(1)
            elif negativeReviewProbability > positiveReviewProbability:
                prediction.append(0)

        return np.array(prediction)
    
    def evaluate(self, y_true, y_predicted):
        accuracy = accuracy_score(y_true, y_predicted)
        print("Accuracy:", accuracy)

        precision = precision_score(y_true, y_predicted)
        print("Precision:", precision)

        # Compute recall
        recall = recall_score(y_true, y_predicted)
        print("Recall:", recall)

        # Compute F1 score
        f1 = f1_score(y_true, y_predicted)
        print("F1 Score:", f1)
        #print("Threshold: "+str(self.threshold))
        return np.array([precision,recall])
    

    


In [16]:
NB = NaiveBayes()
NB.fit(x_train,y_train)
y_predicted = NB.predict(x_test, y_test)
# fit(x_train,y_train)
 
# y_predicted = LogReg.predict(x_test)

NB.evaluate(y_test,y_predicted)

C:\Users\Sophia\AppData\Local\Temp\ipykernel_19124\3336605343.py:61: RuntimeWarning: overflow encountered in multiply
  negativeReviewProbability = round(negativeReviewProbability * self.boundProbabilityC0[y], 2)
C:\Users\Sophia\AppData\Local\Temp\ipykernel_19124\3336605343.py:65: RuntimeWarning: overflow encountered in multiply
  positiveReviewProbability = round(positiveReviewProbability * self.boundProbabilityC1[y], 2)
C:\Users\Sophia\AppData\Local\Temp\ipykernel_19124\3336605343.py:63: RuntimeWarning: overflow encountered in multiply
  negativeReviewProbability = round(negativeReviewProbability * (1 - self.boundProbabilityC0[y]), 2)
C:\Users\Sophia\AppData\Local\Temp\ipykernel_19124\3336605343.py:67: RuntimeWarning: overflow encountered in multiply
  positiveReviewProbability = round(positiveReviewProbability * (1 - self.boundProbabilityC1[y]), 2)


Accuracy: 0.49596
Precision: 0.4960075895327694
Recall: 0.50192
F1 Score: 0.4989462801701857


array([0.49600759, 0.50192   ])